In [3]:

from utils.trainfiles import TrainFiles
from utils.dataloader import DataLoader
from model.unet import UNet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    print('GPU ready')
else:
    print('Warning: only CPU found')

GPU ready


# Generate trainfiles.json

In [2]:
trainfiles = TrainFiles('trainfiles.json',True)
trainfiles.find_files('/home/stephan/Desktop/',['.stk'])

# Train UNet

In [8]:
trainfiles = TrainFiles('trainfiles.json',False)
dataloader = DataLoader(trainfiles,batch_size=6,n_pre=2,n_post=2,train_height=400,train_width=400,load_multiple_targets_per_file=True,n_multiple_targets=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNet(dataloader.n_pre + dataloader.n_post).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.L1Loss()


In [9]:
# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    i = 0
    while not dataloader.epoch_done:
        batch_generated = dataloader.get_batch()
        if not batch_generated:
            break
        data = dataloader.X.to(device)
        targets = dataloader.y.to(device)
        model.train()
        outputs = model(data)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(f'Batch {i+1} (samples {(i+1)*dataloader.batch_size}), Loss: {loss.item()}')
        i += 1
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    dataloader.shuffle_array()

Batch 1 (samples 6), Loss: 0.6909609436988831
Batch 11 (samples 66), Loss: 0.27553391456604004
Batch 21 (samples 126), Loss: 0.2531118392944336
Batch 31 (samples 186), Loss: 0.26683464646339417
Batch 41 (samples 246), Loss: 0.31963130831718445
Batch 51 (samples 306), Loss: 0.24245847761631012
Batch 61 (samples 366), Loss: 0.22504113614559174
Batch 71 (samples 426), Loss: 0.20224836468696594
Batch 81 (samples 486), Loss: 0.23596012592315674
Batch 91 (samples 546), Loss: 0.25659382343292236
Batch 101 (samples 606), Loss: 0.2831600308418274
